In [ ]:
import sys
import os
sys.path.insert(0, os.path.abspath('..'))

import pytest
import json
import csv
import tempfile
from pathlib import Path
from core.common.report_exporter import ReportExporter

## Test Initialization

In [ ]:
def test_report_exporter_init():
    """Test ReportExporter initialization"""
    with tempfile.TemporaryDirectory() as tmpdir:
        exporter = ReportExporter(output_dir=tmpdir)
        
        assert exporter.output_dir == Path(tmpdir)
        assert exporter.output_dir.exists()
        print("✅ Initialization test passed")

test_report_exporter_init()

## Test JSON Export

In [ ]:
def test_export_json():
    """Test JSON export functionality"""
    with tempfile.TemporaryDirectory() as tmpdir:
        exporter = ReportExporter(output_dir=tmpdir)
        
        test_data = {
            'cpu': 50.0,
            'memory': 60.0,
            'disk': 70.0
        }
        
        filepath = exporter.export_json(test_data, 'test_report')
        
        # Verify file exists
        assert Path(filepath).exists()
        
        # Verify content
        with open(filepath, 'r') as f:
            loaded_data = json.load(f)
        
        assert loaded_data['cpu'] == 50.0
        assert loaded_data['memory'] == 60.0
        print(f"✅ JSON export test passed: {filepath}")

test_export_json()

## Test CSV Export

In [ ]:
def test_export_csv():
    """Test CSV export functionality"""
    with tempfile.TemporaryDirectory() as tmpdir:
        exporter = ReportExporter(output_dir=tmpdir)
        
        test_data = [
            {'metric': 'cpu', 'value': 50.0},
            {'metric': 'memory', 'value': 60.0}
        ]
        
        filepath = exporter.export_csv(test_data, 'test_report')
        
        # Verify file exists
        assert Path(filepath).exists()
        
        # Verify content
        with open(filepath, 'r', newline='') as f:
            reader = csv.DictReader(f)
            rows = list(reader)
        
        assert len(rows) == 2
        assert rows[0]['metric'] == 'cpu'
        print(f"✅ CSV export test passed: {filepath}")

test_export_csv()

## Test Text Export

In [ ]:
def test_export_text():
    """Test text export functionality"""
    with tempfile.TemporaryDirectory() as tmpdir:
        exporter = ReportExporter(output_dir=tmpdir)
        
        test_content = "System Report\nCPU: 50.0%\nMemory: 60.0%"
        
        filepath = exporter.export_text(test_content, 'test_report')
        
        # Verify file exists
        assert Path(filepath).exists()
        
        # Verify content
        with open(filepath, 'r') as f:
            content = f.read()
        
        assert 'System Report' in content
        assert 'CPU: 50.0%' in content
        print(f"✅ Text export test passed: {filepath}")

test_export_text()

## Test Full Report Export

In [ ]:
def test_export_full_report():
    """Test full report export (requires psutil)"""
    with tempfile.TemporaryDirectory() as tmpdir:
        exporter = ReportExporter(output_dir=tmpdir)
        
        try:
            # Test JSON format
            filepath = exporter.export_full_report('json')
            assert Path(filepath).exists()
            
            # Verify it's valid JSON
            with open(filepath, 'r') as f:
                data = json.load(f)
            
            assert 'timestamp' in data
            assert 'system' in data
            print(f"✅ Full report export test passed: {filepath}")
            
        except Exception as e:
            print(f"⚠️ Full report test skipped (psutil may not be available): {e}")

test_export_full_report()

## Test Filename Generation

In [ ]:
def test_filename_timestamp():
    """Test that filenames include timestamps"""
    with tempfile.TemporaryDirectory() as tmpdir:
        exporter = ReportExporter(output_dir=tmpdir)
        
        filepath1 = exporter.export_json({'test': 1}, 'report')
        filepath2 = exporter.export_json({'test': 2}, 'report')
        
        # Filenames should be different (timestamps)
        assert filepath1 != filepath2
        assert 'report_' in filepath1
        print(f"✅ Filename timestamp test passed")

test_filename_timestamp()

## Run All Tests

In [ ]:
# Run all tests
pytest.main(['-v', '--tb=short', __file__])